# Brian AutoEncoder1

### Importing Our Required Packages and things

In [1]:
#importing required libraries and packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import ExcelWriter
from sklearn import metrics
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm

#set plot style to 'ggplot' and don't limit the view of DF when diaplyhign them to screen
plt.style.use('ggplot')
pd.options.display.max_columns = None

### Read the Data In

In [2]:
#importing our data and resetting our index
df = pd.read_excel('Data/Comp2_Raw_Data.xls')
df.head(1)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1


### Renaming and Dropping Columns

To make our dataset cleaner, we renamed our columns and we also dropped the ID column because it had no value to our model

In [3]:
df = df.drop(columns = ['ID'])
df.columns = ['Credit_Limit', 'Gender', 'Education', 'Marriage',  'Age', 'Pay_Sept', 'Pay_Aug', 'Pay_Jul', 'Pay_Jun', 'Pay_May', 'Pay_Apr', 
             'Bill_Amt_Sept', 'Bill_Amt_Aug', 'Bill_Amt_Jul', 'Bill_Amt_Jun', 'Bill_Amt_May', 'Bill_Amt_Apr','Pay_Amt_Sept', 'Pay_Amt_Aug',
             'Pay_Amt_Jul', 'Pay_Amt_Jun', 'Pay_Amt_May', 'Pay_Amt_Apr', 'Default']
df.head(5)

,Credit_Limit,Gender,Education,Marriage,Age,Pay_Sept,Pay_Aug,Pay_Jul,Pay_Jun,Pay_May,Pay_Apr,Bill_Amt_Sept,Bill_Amt_Aug,Bill_Amt_Jul,Bill_Amt_Jun,Bill_Amt_May,Bill_Amt_Apr,Pay_Amt_Sept,Pay_Amt_Aug,Pay_Amt_Jul,Pay_Amt_Jun,Pay_Amt_May,Pay_Amt_Apr,Default
0,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


### Creating A Target DF and a Feature DF 

We seperated our data into a df_target which held all of our target variables. This way we don't accidently scale/transform them or include them as a feature in our feature selection/reduction  below

In [4]:
#Copy our target variables to their own df
df_target = df[['Default']].copy()
df_target.head(10)
#change the data type to categorical
df_target['Default'] = pd.Categorical(df_target.Default)
#Drop Default from our target variable df
df = df.drop(['Default'], axis=1)
df.head(5)

,Credit_Limit,Gender,Education,Marriage,Age,Pay_Sept,Pay_Aug,Pay_Jul,Pay_Jun,Pay_May,Pay_Apr,Bill_Amt_Sept,Bill_Amt_Aug,Bill_Amt_Jul,Bill_Amt_Jun,Bill_Amt_May,Bill_Amt_Apr,Pay_Amt_Sept,Pay_Amt_Aug,Pay_Amt_Jul,Pay_Amt_Jun,Pay_Amt_May,Pay_Amt_Apr
0,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0
1,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000
2,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000
3,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000
4,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679


In [5]:
#chacking that our data was transfered properly.
df_target.head(5)

,Default
0,1
1,1
2,0
3,0
4,0


### Changing Our DataTypes

We want to make our fields from integer to float so we do that here

In [6]:
#chnage column datatypes to float
for col in df:
    df[col]=pd.to_numeric(df[col], errors='coerce', downcast='float')
df.dtypes

Credit_Limit     float32
Gender           float32
Education        float32
Marriage         float32
Age              float32
Pay_Sept         float32
Pay_Aug          float32
Pay_Jul          float32
Pay_Jun          float32
Pay_May          float32
Pay_Apr          float32
Bill_Amt_Sept    float32
Bill_Amt_Aug     float32
Bill_Amt_Jul     float32
Bill_Amt_Jun     float32
Bill_Amt_May     float32
Bill_Amt_Apr     float32
Pay_Amt_Sept     float32
Pay_Amt_Aug      float32
Pay_Amt_Jul      float32
Pay_Amt_Jun      float32
Pay_Amt_May      float32
Pay_Amt_Apr      float32
dtype: object

## Debt Per Month

In [7]:
df['Debt_Apr'] = df['Bill_Amt_Apr'] - df['Pay_Amt_Apr']
df['Debt_May'] = df['Bill_Amt_May'] - df['Pay_Amt_May']
df['Debt_Jun'] = df['Bill_Amt_Jun'] - df['Pay_Amt_Jun']
df['Debt_Jul'] = df['Bill_Amt_Jul'] - df['Pay_Amt_Jul']
df['Debt_Aug'] = df['Bill_Amt_Aug'] - df['Pay_Amt_Aug']
df['Debt_Sept'] = df['Bill_Amt_Sept'] - df['Pay_Amt_Sept']

In [8]:
df.head()

,Credit_Limit,Gender,Education,Marriage,Age,Pay_Sept,Pay_Aug,Pay_Jul,Pay_Jun,Pay_May,Pay_Apr,Bill_Amt_Sept,Bill_Amt_Aug,Bill_Amt_Jul,Bill_Amt_Jun,Bill_Amt_May,Bill_Amt_Apr,Pay_Amt_Sept,Pay_Amt_Aug,Pay_Amt_Jul,Pay_Amt_Jun,Pay_Amt_May,Pay_Amt_Apr,Debt_Apr,Debt_May,Debt_Jun,Debt_Jul,Debt_Aug,Debt_Sept
0,20000.0,2.0,2.0,1.0,24.0,2.0,2.0,-1.0,-1.0,-2.0,-2.0,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,689.0,2413.0,3913.0
1,120000.0,2.0,2.0,2.0,26.0,-1.0,2.0,0.0,0.0,0.0,2.0,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1261.0,3455.0,2272.0,1682.0,725.0,2682.0
2,90000.0,2.0,2.0,2.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,10549.0,13948.0,13331.0,12559.0,12527.0,27721.0
3,50000.0,2.0,2.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,28547.0,27890.0,27214.0,48091.0,46214.0,44990.0
4,50000.0,1.0,2.0,1.0,57.0,-1.0,0.0,-1.0,0.0,0.0,0.0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,18452.0,18457.0,11940.0,25835.0,-31011.0,6617.0


In [9]:
df_ac = df_target[['Default']].copy()

In [10]:
df_ac['Debt_Apr'] = df['Debt_Apr']
df_ac['Debt_May'] = df['Debt_May']
df_ac['Debt_Jun'] = df['Debt_Jun']
df_ac['Debt_Jul'] = df['Debt_Jul']
df_ac['Debt_Aug'] = df['Debt_Aug']
df_ac['Debt_Sept'] = df['Debt_Sept']

In [11]:
df_ac['Debt_Apr_May'] = 'null'
df_ac['Debt_Apr_May'][df_ac['Debt_Apr']-df_ac['Debt_May']<0] =1
df_ac['Debt_Apr_May'][df_ac['Debt_Apr']-df_ac['Debt_May']==0] =0
df_ac['Debt_Apr_May'][df_ac['Debt_Apr']-df_ac['Debt_May']>1] =-1

df_ac['Debt_May_Jun'] = 'null'
df_ac['Debt_May_Jun'][df_ac['Debt_May']-df_ac['Debt_Jun']<0] =1
df_ac['Debt_May_Jun'][df_ac['Debt_May']-df_ac['Debt_Jun']==0] =0
df_ac['Debt_May_Jun'][df_ac['Debt_May']-df_ac['Debt_Jun']>1] =-1

df_ac['Debt_Jun_Jul'] = 'null'
df_ac['Debt_Jun_Jul'][df_ac['Debt_Jun']-df_ac['Debt_Jul']<0] =1
df_ac['Debt_Jun_Jul'][df_ac['Debt_Jun']-df_ac['Debt_Jul']==0] =0
df_ac['Debt_Jun_Jul'][df_ac['Debt_Jun']-df_ac['Debt_Jul']>1] =-1

df_ac['Debt_Jul_Aug'] = 'null'
df_ac['Debt_Jul_Aug'][df_ac['Debt_Jul']-df_ac['Debt_Aug']<0] =1
df_ac['Debt_Jul_Aug'][df_ac['Debt_Jul']-df_ac['Debt_Aug']==0] =0
df_ac['Debt_Jul_Aug'][df_ac['Debt_Jul']-df_ac['Debt_Aug']>1] =-1

df_ac['Debt_Aug_Sept'] = 'null'
df_ac['Debt_Aug_Sept'][df_ac['Debt_Aug']-df_ac['Debt_Sept']<0] =1
df_ac['Debt_Aug_Sept'][df_ac['Debt_Aug']-df_ac['Debt_Sept']==0] =0
df_ac['Debt_Aug_Sept'][df_ac['Debt_Aug']-df_ac['Debt_Sept']>1] =-1


/opt/tljh/user/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/tljh/user/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/tljh/user/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/tljh/user/lib/python3.6/

In [12]:
df_ac = df_ac.drop(['Debt_Apr', 'Debt_May', 'Debt_Jun', 'Debt_Jul', 'Debt_Aug', 'Debt_Sept'], axis=1)

In [13]:
df_ac.head()

,Default,Debt_Apr_May,Debt_May_Jun,Debt_Jun_Jul,Debt_Jul_Aug,Debt_Aug_Sept
0,1,0,0,1,1,1
1,1,1,-1,-1,-1,1
2,0,1,-1,-1,-1,1
3,0,-1,-1,1,-1,-1
4,0,1,-1,1,-1,1


In [14]:
for col in df_ac:
    df_ac[col]=pd.to_numeric(df_ac[col], errors='coerce', downcast='float')

In [15]:
df_ac = df_ac.dropna()

In [16]:
df_ac.isna().sum()

Default          0
Debt_Apr_May     0
Debt_May_Jun     0
Debt_Jun_Jul     0
Debt_Jul_Aug     0
Debt_Aug_Sept    0
dtype: int64

In [17]:
cols = list(df_ac.columns)
cols

['Default',
 'Debt_Apr_May',
 'Debt_May_Jun',
 'Debt_Jun_Jul',
 'Debt_Jul_Aug',
 'Debt_Aug_Sept']

In [18]:
X = df_ac[cols[1:]]
X.head()

,Debt_Apr_May,Debt_May_Jun,Debt_Jun_Jul,Debt_Jul_Aug,Debt_Aug_Sept
0,0.0,0.0,1.0,1.0,1.0
1,1.0,-1.0,-1.0,-1.0,1.0
2,1.0,-1.0,-1.0,-1.0,1.0
3,-1.0,-1.0,1.0,-1.0,-1.0
4,1.0,-1.0,1.0,-1.0,1.0


In [19]:
y = df_ac.Default

In [20]:
y.head()

0    1.0
1    1.0
2    0.0
3    0.0
4    0.0
Name: Default, dtype: float32

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=2019)

In [22]:
#Create a svm Classifier
clf = svm.SVC(kernel='linear', random_state=2019) # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [23]:
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7760546940136736


In [24]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(23985, 5)
(23985,)
(5997, 5)
(5997,)


In [ ]:
# Begin oversampling
oversample = pd.concat([X_train,y_train],axis=1)
max_size = oversample['Default'].value_counts().max()
lst = [oversample]

for class_index, group in oversample.groupby('Default'):
    lst.append(group.sample(max_size-len(group), replace=True))
X_train = pd.concat(lst)
y_train=pd.DataFrame.copy(X_train['Default'])
del X_train['Default']

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [25]:
#Create a svm Classifier
clf = svm.SVC(kernel='linear', random_state=2019) # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [26]:
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7760546940136736


In [27]:
# import required package for data handling
import pandas as pd
import numpy as np

# import required packages for splitting data
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# import required packages for evaluating models
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# import `logistic regression` model
from sklearn.linear_model import LogisticRegression

In [28]:
# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []


#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,10):
    print(count)
    #Model building
    clf1 = svm.SVC(kernel='linear', random_state=2019)

    
    # Splitting data into testing and training
    X2_train, X2_test, y2_train, y2_test = train_test_split(X, y, test_size=0.2, random_state=123)
    
    # Begin oversampling
    oversample = pd.concat([X2_train,y2_train],axis=1)
    max_size = oversample['Default'].value_counts().max()
    lst = [oversample]
    for class_index, group in oversample.groupby('Default'):
        lst.append(group.sample(max_size-len(group), replace=True))
    X2_train = pd.concat(lst)
    y2_train=pd.DataFrame.copy(X2_train['Default'])
    del X2_train['Default']
    
    # fitting model on oversampled data
    clf1.fit(X2_train, y2_train)
    
    y2_pred = clf1.predict(X2_test)
    
    
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=123)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(clf1, X2_train, y2_train, cv=kfold, scoring=scoring)
    
    #calculate f1-score and AUC
    
    clf1_roc_auc = roc_auc_score(y2_test, y2_pred)
    
    
    #calculate average f1-score and AUC
    f1_score_lst.append(precision_recall_fscore_support(y2_test, y2_pred, average='weighted')[2])
    auc_lst.append(clf1_roc_auc)
    
    
print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

confusion_matrix_y2 = confusion_matrix(y2_test, y2_pred)


print('Accuracy of classifier on test set: {:.3f}'.format(clf1.score(X2_test, y2_test)))

print("10-fold cross validation average accuracy of clf1: %.3f" % (results.mean()))

print('Confusion Matrix for Classfier:')
print(confusion_matrix_y2)

print('Classification Report for Classfier:')
print(classification_report(y2_test, y2_pred))




1
2
3
4
5
6
7
8
9
F1 0.5626; AUC 0.5168 
Accuracy of classifier on test set: 0.566
10-fold cross validation average accuracy of clf1: 0.181
Confusion Matrix for Classfier:
[[2883 1806]
 [ 797  511]]
Classification Report for Classfier:
             precision    recall  f1-score   support

        0.0       0.78      0.61      0.69      4689
        1.0       0.22      0.39      0.28      1308

avg / total       0.66      0.57      0.60      5997



In [29]:
df_ac.head()

,Default,Debt_Apr_May,Debt_May_Jun,Debt_Jun_Jul,Debt_Jul_Aug,Debt_Aug_Sept
0,1.0,0.0,0.0,1.0,1.0,1.0
1,1.0,1.0,-1.0,-1.0,-1.0,1.0
2,0.0,1.0,-1.0,-1.0,-1.0,1.0
3,0.0,-1.0,-1.0,1.0,-1.0,-1.0
4,0.0,1.0,-1.0,1.0,-1.0,1.0


In [30]:
df_ac.to_csv('Output.csv', sep=',')